In [1]:
# %pip install langchain fastembed qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 702.8 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 2.7 MB/s eta 0:00:0000:0100:01
  Using cached snowballstemmer-2.2.0-py2.py3-none-any.whl (93 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-macosx_11_0_arm64.whl size=172717 sha256=71ede1728c9acc96edd1b728df8cc19f46859dab8a48f8d6d784493e3267d017
  Stored in directory: /Users/devanshu/Library/Caches/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built PyStemmer
  Attempting uninstall: pillow
    Found existing installation: pillow 10.2.0
    Uninstalling pillow-10.2.0:
      Successfully uninstalled pillow-10.2.0
ERROR: pip's dependency resolver does not currently take into

In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from qdrant_client import QdrantClient
from qdrant_client.http import models


import uuid
import time
from typing import List
from fastembed import TextEmbedding
from qdrant_client.http.models import PointStruct, SearchParams

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain_groq import ChatGroq


In [2]:
qdrant_uri = "<ENTER YOUR QDRANT URI HERE>"
qdrant_api = "<ENTER YOUR QDRANT API HERE>"
groq_key = "<ENTER YOUR GROQ API HERE>"

In [3]:
llm = ChatGroq(groq_api_key=groq_key,
               model='llama3-8b-8192',
               temperature=0.2) # set temperature by your own

In [4]:
data = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split="train")
data = data.to_pandas()
data.head()

,instruction,input,output
0,"If you are a doctor, please answer the medical...",I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,"If you are a doctor, please answer the medical...",My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"If you are a doctor, please answer the medical...","Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,"If you are a doctor, please answer the medical...",lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,"If you are a doctor, please answer the medical...",I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...


In [5]:
MAX_ROWS = 1000
OUTPUT = "output"
subset_data = data.head(MAX_ROWS)

In [6]:
client = QdrantClient(
    qdrant_uri,
    api_key=qdrant_api
)


chunks = subset_data[OUTPUT].to_list()

# Add data to defined collection
client.add(
   collection_name='ChatDoctor',
   documents=chunks
)





['5e1c01d4540c44039e2bc48ab12ed8cc',
 'd876b922b65f4d38af6ba0589ab85129',
 '2aa8c0d81979448fb1baed435b2b3215',
 'c6255352efad49c18edab7f71665640d',
 'ae6d591ec8e54943a308f449182ecd49',
 '2dd4f7fc33be44be954a81aa5d2333c0',
 '175943b1a84b4f5dbca3deefa379c1fd',
 'a67e8ebd9b014e5586a718080ea2670b',
 'fd35624e421e4686a2c6efe7e21be890',
 '7644f2a1fb4b4c198bb7a8fcec176b61',
 'fb84a2204d2f463996ffe65ac462ca38',
 '68392ba8921f4caba1d698e8b2db1545',
 '35c3ac80401a4363a1b915a408f3c599',
 '64e6d108018e4a2dac9a8ff9188d694d',
 'ad2ad1fe7ac84c73ba7cf218e15601fb',
 '843f45c7e316463cb3c9f7a2b5acb675',
 'a207a2c319594a55936c2e92daba32ea',
 'bfda38aad9d140d08a98834717453b5b',
 '955732982cb042fcb1fa14567ddaf612',
 'a98e3b43ec4746839c61af040ab134ca',
 'c3035be9b41d4015a4e9131dd7cb32da',
 'd3ca30c6d011402980d1164372a8d589',
 '48fffe8904934de5a00383620a5ccdfe',
 'c0a12db3b58b43629d54e2704521a62d',
 '3ccb7fdacb374d18922e3bf351231019',
 'a1251ea75ac24940a58877c3486ea0fb',
 '7ab2534009a74a98a3b87be4621da21e',
 

In [7]:
threshold = 0.87

In [8]:
class QdrantVectorStore():
    def __init__(self, qdrant_uri, qdrant_api, threshold):
        self.encoder = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")
        self.client = QdrantClient()
        self.db_collection_name = "ChatDoctor"
        
        self.db_client = QdrantClient(
            qdrant_uri,
            api_key=qdrant_api
        )
        
        self.euclidean_threshold = threshold
   
    def get_embedding(self, question):
        embedding = list(self.encoder.embed(question))[0]
        return embedding
   
    def query_database(self, query_text):
        result = self.db_client.query(
            query_text=query_text,
            limit=3,
            collection_name=self.db_collection_name
        )
        return result
   
    def query(self, question):
        start_time = time.time()
        
        db_results = self.query_database(question)
        
        if db_results:
            response_text = db_results[0].document
            print('Retrieval without cache.')
            elapsed_time = time.time() - start_time
            print(f"Time taken: {elapsed_time:.3f} seconds")
            return response_text
        
        print("No answer found in Database")
        elapsed_time = time.time() - start_time
        print(f"Time taken: {elapsed_time:.3f} seconds")
        return "No answer available"


In [9]:
vector_db = QdrantVectorStore(qdrant_uri, qdrant_api, threshold)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
def chat_query(question):
    context = vector_db.query(question)
   
    prompt_template = """Answer the question with given context for the medical question. Please ensure that the answer is relevant to the question.
                In case the original context is not relevant, please answer the question as per the question only.
                {context}
           
                Question: {question}"""
    prompt = PromptTemplate.from_template(prompt_template)


    chain = prompt | llm | StrOutputParser()
   
    result = chain.invoke({"question":question, "context":context})
    # print("\n\n")
    return result

In [23]:
question_2 = "I have a sore throat."
result = chat_query(question_2)

Retrieval without cache.
Time taken: 1.497 seconds


In [24]:
from IPython.display import Markdown
Markdown(result)

Based on the given context, it seems that your sore throat might be accompanied by other symptoms such as a runny or stuffy nose, feeling of liquid running down the back of your throat, frequent throat clearing, and hoarseness. In this case, I would recommend trying the medication regimen suggested, which includes a tablet containing montelukast + levocetirizine, cough suppressant syrup before bed, and pantoprazole 40mg empty stomach in the morning. Additionally, you may want to consider avoiding allergens and reducing acid reflux by eating smaller, more frequent meals, waiting three to four hours after a meal before lying down, and elevating the head of the bed at night. If your symptoms do not subside, it's recommended to see a doctor and get a chest X-ray done to get a confirmed diagnosis.

In [13]:
vector = vector_db.get_embedding(result)
vector_size = len(vector.tolist())  # Get the size of your vector


In [14]:
print(f"Vector size: {vector_size}")

Vector size: 384


In [35]:
from qdrant_client.http import models

# Define your QdrantVectorStore class and instantiate it
vector_db = QdrantVectorStore(qdrant_uri, qdrant_api, threshold)

# Define the function to perform the insertion
def insert_data_to_qdrant(question, response):
    # Generate embeddings using your TextEmbedding model
    data = question + response
    data_embedding = vector_db.get_embedding(data)
    
    # Prepare the data point for insertion
    point = models.PointStruct(
        id=str(uuid.uuid4()),  # Generate a unique ID (example using UUID)
        payload={
            "question": question,
            "response": response
        },
        vector=data_embedding.tolist()  # Convert embedding to list format
    )
    
    # Upload the point to Qdrant
    try:
        result = client.upload_points(
            collection_name="ChatDoctor",
            points=[point],
            parallel=4,
            max_retries=3
        )
        print("Data point uploaded successfully to Qdrant.")
        return point  # Return the point object after successful insertion
    except Exception as e:
        print(f"Error uploading data point to Qdrant: {e}")
        return None

# Example usage:
question = "test 223``1"
response = chat_query(question)

# Insert the question and response into Qdrant
inserted_point = insert_data_to_qdrant(question, response)

# Print ID and payload key-value pairs if insertion was successful
if inserted_point:
    print(f"Inserted ID: {inserted_point.id}")
    print(f"Payload:")
    print(f"  Question: {inserted_point.payload['question']}")
    print(f"  Response: {inserted_point.payload['response']}")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Retrieval without cache.
Time taken: 1.327 seconds
Data point uploaded successfully to Qdrant.
Inserted ID: df3329f5-45d8-41a7-a484-6258c6658e7a
Payload:
  Question: test 223``1
  Response: Based on the context, it appears that the question is asking for the results of a specific blood test, which is test 223.
